In [ ]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup

from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time

In [ ]:
def initialize_driver(pagenum):
  # Set webdriver options
  options = webdriver.ChromeOptions()
  options.add_argument('--no-sandbox')
  options.add_argument('ignore-certificate-errors')

  driver = webdriver.Chrome(options=options)
  driver.get(f"https://www.99.co/id/jual/rumah?hlmn={pagenum}")
  return driver 

In [ ]:
driver = initialize_driver(1)

In [ ]:
daftar_waktu_update = []
daftar_alamat = []
daftar_jumlah_kamar_tidur = []
daftar_jumlah_kamar_mandi = []
daftar_meter_persegi_bangunan = []
daftar_meter_persegi_tanah = []
daftar_harga = []

In [ ]:
def check_captcha(driver):
  try:
    driver.find_element(By.XPATH, "//*[contains(text(), 'Verify you are human by completing the action below')]")
    return True
  except:
    return False

In [ ]:
pagenum = 1

while True:
    time.sleep(2)
        
    my_page = driver.page_source
    my_html = BeautifulSoup(my_page, "html.parser") # get html
    
    first_containers = my_html.findAll('div', {'class':'cardSecondary__info-wrapper_detail'}) # get containers

    for container in first_containers:
        
        # Extract updated date
        tag_waktu_update = container.find('time')
        waktu_update = tag_waktu_update.get_text().strip() if tag_waktu_update else "unknown"
        daftar_waktu_update = [].append(waktu_update)
        
        # Extract address
        tag_alamat = container.find('address')
        alamat = tag_alamat.get_text().strip() if tag_alamat else "unknown"
        daftar_alamat.append(alamat)
        
        # Extract harga
        price_tag = container.find('div', class_='price__tag')
        price = price_tag.strong.get_text().strip()[2:] if price_tag else "unknown"
        daftar_harga.append(price)
        
    second_containers = my_html.findAll('div', {'class':'cardSecondary__info-wrapper_attributes'}) # get containers

    for container in second_containers:
        # Extract number of bedrooms
        tag_jumlah_kamar_tidur = container.find('div', class_='left').find_all('div')[0]
        jumlah_kamar_tidur = tag_jumlah_kamar_tidur.get_text().strip() if tag_jumlah_kamar_tidur else "unknown"
        daftar_jumlah_kamar_tidur.append(jumlah_kamar_tidur)
        
        # Extract number of bathrooms
        tag_jumlah_kamar_mandi = container.find('div', class_='left').find_all('div')[1]
        jumlah_kamar_mandi = tag_jumlah_kamar_mandi.get_text().strip() if tag_jumlah_kamar_mandi else "unknown"
        daftar_jumlah_kamar_mandi.append(jumlah_kamar_mandi)

        # Extract land area (LT)
        tag_luas_tanah = container.find('div', class_='right').find_all('div')[0]
        luas_tanah = tag_luas_tanah.get_text().strip()[2:-3] if tag_luas_tanah else "unknown"
        daftar_meter_persegi_tanah.append(luas_tanah)

        # Extract building area (LB)
        tag_luas_bangunan = container.find('div', class_='right').find_all('div')[1]
        luas_bangunan = tag_luas_bangunan.get_text().strip()[2:-3] if tag_luas_bangunan else "unknown"
        daftar_meter_persegi_bangunan.append(luas_bangunan)

    try: 
        if check_captcha(driver):
            print("CAPTCHA detected. Close.")
            driver.quit()
            time.sleep(60)
            initialize_driver(pagenum+1)
            
        # Wait for the "Next" button to be clickable
        next_button = driver.find_element(By.CLASS_NAME, 'ui-atomic-pagination__item--next')
        
        # Scroll to the "Next" button and ensure it's in the middle of the screen
        driver.execute_script("window.scrollTo({ top: arguments[0].getBoundingClientRect().top + window.scrollY - (window.innerHeight / 2), behavior: 'smooth' });", next_button)
        
        # Click the "Next" button using JavaScript to ensure interaction
        driver.execute_script("arguments[0].click();", next_button)
        
        pagenum += 1
    
        print('Page:', pagenum)

    except Exception as e:
        print(e)
        print(f"No more pages. Exiting. Error: {e}")
        break
    
print('End of loop')

In [ ]:
# Create a DataFrame from the lists
df = pd.DataFrame({
    'updated_time': pd.Series(daftar_waktu_update),
    'address': pd.Series(daftar_alamat),
    'bedrooms': pd.Series(daftar_jumlah_kamar_tidur),
    'bathrooms': pd.Series(daftar_jumlah_kamar_mandi),
    'building_area_sqm': pd.Series(daftar_meter_persegi_bangunan),
    'land_area_sqm': pd.Series(daftar_meter_persegi_tanah),
    'price': pd.Series(daftar_harga),
})

# Save the DataFrame to a CSV file
df.to_csv('property_listings.csv', index=False)

print("Data saved to property_listings.csv")